# Tutorial Pandas: Grouping Data

Selamat datang di tutorial Pandas untuk Grouping Data! Dalam tutorial ini, kita akan mempelajari cara mengelompokkan dan mengagregasi data menggunakan Pandas.

## Daftar Isi:
1. Pengenalan Grouping
2. Metode-metode Grouping
   - groupby()
   - agg()
   - transform()
3. Fungsi Agregasi
4. Multiple Grouping
5. Studi Kasus
6. Latihan

## 1. Pengenalan Grouping

Grouping adalah teknik yang sangat penting dalam analisis data untuk mengelompokkan data berdasarkan satu atau lebih kriteria, kemudian melakukan perhitungan atau agregasi pada kelompok-kelompok tersebut. 

Mari kita mulai dengan mengimpor library yang diperlukan dan membuat data contoh:

In [31]:
# Import library yang diperlukan
import pandas as pd
import numpy as np

# Membuat data penjualan toko
np.random.seed(42)
n = 50

data_penjualan = {
    'Tanggal': pd.date_range(start='2024-01-01', periods=n),
    'Produk': np.random.choice(['Laptop', 'Smartphone', 'Tablet', 'Smartwatch'], n),
    'Kategori': np.random.choice(['Elektronik', 'Gadget'], n),
    'Jumlah': np.random.randint(1, 10, n),
    'Harga': np.random.choice([500, 800, 1000, 1500, 2000], n),
    'Kota': np.random.choice(['Jakarta', 'Bandung', 'Surabaya'], n)
}

# Membuat DataFrame
df = pd.DataFrame(data_penjualan)

# Menambahkan kolom Total
df['Total'] = df['Jumlah'] * df['Harga']

print("Data Penjualan:")
print(df.head(10))
print("\nInformasi Dataset:")
print(df.info())

Data Penjualan:
     Tanggal      Produk    Kategori  Jumlah  Harga      Kota  Total
0 2024-01-01      Tablet  Elektronik       2   1000  Surabaya   2000
1 2024-01-02  Smartwatch      Gadget       2    500  Surabaya   1000
2 2024-01-03      Laptop  Elektronik       4    800   Bandung   3200
3 2024-01-04      Tablet      Gadget       8    800   Jakarta   6400
4 2024-01-05      Tablet  Elektronik       7   1500   Jakarta  10500
5 2024-01-06  Smartwatch      Gadget       9   2000   Bandung  18000
6 2024-01-07      Laptop      Gadget       8   1000   Jakarta   8000
7 2024-01-08      Laptop  Elektronik       5    500   Bandung   2500
8 2024-01-09      Tablet  Elektronik       2   1500   Bandung   3000
9 2024-01-10  Smartphone  Elektronik       5   2000  Surabaya  10000

Informasi Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Tangga

## 2. Metode-metode Grouping

### 2.1 Groupby Dasar

`groupby()` adalah metode utama untuk mengelompokkan data di Pandas. Mari kita lihat beberapa contoh dasar:

In [33]:
# 1. Groupby satu kolom
print("Rata-rata penjualan per Kota:")
print(df.groupby('Kota')['Total'].mean().round(2))
print("\n" + "-"*50)

# 2. Menghitung beberapa statistik sekaligus
print("\nStatistik penjualan per Produk:")
stats = df.groupby('Produk').agg({
    'Jumlah': 'sum',
    'Total': ['mean', 'sum'],
    'Harga': 'mean'
}).round(2)
print(stats)
print("\n" + "-"*50)

# 3. Menghitung jumlah transaksi per kategori dan kota
print("\nJumlah transaksi per Kategori dan Kota:")
print(pd.crosstab(df['Kategori'], df['Kota']))

Rata-rata penjualan per Kota:
Kota
Bandung     7064.71
Jakarta     7317.65
Surabaya    6006.25
Name: Total, dtype: float64

--------------------------------------------------

Statistik penjualan per Produk:
           Jumlah    Total            Harga
              sum     mean     sum     mean
Produk                                     
Laptop         63  5480.00   54800   840.00
Smartphone     41  4627.27   50900  1236.36
Smartwatch     92  7968.75  127500  1250.00
Tablet         72  8261.54  107400  1369.23

--------------------------------------------------

Jumlah transaksi per Kategori dan Kota:
Kota        Bandung  Jakarta  Surabaya
Kategori                              
Elektronik        6        9         6
Gadget           11        8        10


### 2.2 Fungsi Agregasi Kustom

Selain fungsi agregasi bawaan seperti `sum()`, `mean()`, `count()`, kita juga bisa membuat fungsi agregasi kustom:

In [34]:
# Membuat fungsi agregasi kustom
def range_harga(x):
    return x.max() - x.min()

def proporsi_mahal(x):
    return (x > 1000).mean() * 100

# Menggunakan fungsi kustom dalam groupby
hasil_analisis = df.groupby('Kategori').agg({
    'Harga': [
        'mean',
        range_harga,
        proporsi_mahal
    ],
    'Total': 'sum'
}).round(2)

# Merapikan nama kolom
hasil_analisis.columns = ['Rata-rata Harga', 'Range Harga', 'Proporsi Mahal (%)', 'Total Penjualan']

print("Analisis Harga per Kategori:")
print(hasil_analisis)

Analisis Harga per Kategori:
            Rata-rata Harga  Range Harga  Proporsi Mahal (%)  Total Penjualan
Kategori                                                                     
Elektronik          1347.62         1500               57.14           166200
Gadget              1086.21         1500               37.93           174400


### 2.3 Transform dan Window Functions

`transform()` memungkinkan kita untuk melakukan operasi yang mempertahankan ukuran DataFrame asli. Ini sangat berguna untuk perhitungan seperti persentase dari total atau perbedaan dari rata-rata kelompok:

In [36]:
# Menambahkan kolom rata-rata harga per kategori
df['Rata2_Harga_Kategori'] = df.groupby('Kategori')['Harga'].transform('mean')

# Menghitung perbedaan dari rata-rata kategori
df['Selisih_dari_Rata2'] = df['Harga'] - df['Rata2_Harga_Kategori']

# Menghitung persentase dari total penjualan per kota
df['Persen_dari_Total_Kota'] = df.groupby('Kota')['Total'].transform(
    lambda x: x / x.sum() * 100
)

print("Hasil Transform:")
print(df[['Produk', 'Kategori', 'Harga', 'Rata2_Harga_Kategori', 
          'Selisih_dari_Rata2', 'Persen_dari_Total_Kota']].head(10).round(2))

KeyError: 'Kota'

In [37]:
# Menambahkan kolom rata-rata harga per kategori
df['Rata2_Harga_Kategori'] = df.groupby('Kategori')['Harga'].transform('mean')

# Menghitung perbedaan dari rata-rata kategori
df['Selisih_dari_Rata2'] = df['Harga'] - df['Rata2_Harga_Kategori']

print("Analisis Harga Relatif terhadap Kategori:")
print(df[['Produk', 'Kategori', 'Harga', 'Rata2_Harga_Kategori', 
          'Selisih_dari_Rata2']].head(10).round(2))

Analisis Harga Relatif terhadap Kategori:
       Produk    Kategori     Harga  Rata2_Harga_Kategori  Selisih_dari_Rata2
0      Laptop  Elektronik   6923388            9135561.33         -2212173.33
1  Smartphone      Gadget   7050634            8815992.50         -1765358.50
2      Tablet      Gadget  10581351            8815992.50          1765358.50
3   Headphone   Aksesoris   4804572            8398372.00         -3593800.00
4       Mouse   Aksesoris  13815092            8398372.00          5416720.00
5    Keyboard   Aksesoris   2734489            8398372.00         -5663883.00
6     Monitor  Elektronik  10458614            9135561.33          1323052.67
7     Printer  Elektronik  10024682            9135561.33           889120.67
8     Speaker   Aksesoris  12933495            8398372.00          4535123.00
9     Charger   Aksesoris   7704212            8398372.00          -694160.00


### 2.4 Multiple Grouping

Kita juga bisa melakukan grouping berdasarkan multiple kolom:

In [38]:
# Multiple grouping dengan berbagai agregasi
hasil_multi = df.groupby(['Kategori', 'Produk']).agg({
    'Jumlah': ['count', 'sum', 'mean'],
    'Harga': ['min', 'max', 'mean'],
    'Total': 'sum'
}).round(2)

print("Analisis Detail per Kategori dan Produk:")
print(hasil_multi)

KeyError: "Column(s) ['Jumlah', 'Total'] do not exist"

In [39]:
# Multiple grouping dengan berbagai agregasi
hasil_multi = df.groupby(['Kategori', 'Produk']).agg({
    'Harga': ['count', 'min', 'max', 'mean']
}).round(2)

print("Analisis Detail per Kategori dan Produk:")
print(hasil_multi)

Analisis Detail per Kategori dan Produk:
                      Harga                                
                      count       min       max        mean
Kategori   Produk                                          
Aksesoris  Charger        1   7704212   7704212   7704212.0
           Headphone      1   4804572   4804572   4804572.0
           Keyboard       1   2734489   2734489   2734489.0
           Mouse          1  13815092  13815092  13815092.0
           Speaker        1  12933495  12933495  12933495.0
Elektronik Laptop         1   6923388   6923388   6923388.0
           Monitor        1  10458614  10458614  10458614.0
           Printer        1  10024682  10024682  10024682.0
Gadget     Smartphone     1   7050634   7050634   7050634.0
           Tablet         1  10581351  10581351  10581351.0


## 3. Studi Kasus: Analisis Penjualan

Mari kita coba sebuah studi kasus yang lebih kompleks dengan menganalisis data penjualan. Kita akan membuat dataset baru yang lebih lengkap:

In [40]:
# Membuat dataset penjualan yang lebih lengkap
np.random.seed(42)
n_transaksi = 100

data_penjualan = {
    'Tanggal': pd.date_range(start='2024-01-01', periods=n_transaksi),
    'ID_Produk': [f'P{str(i).zfill(3)}' for i in np.random.randint(1, 21, n_transaksi)],
    'Kategori': np.random.choice(['Elektronik', 'Fashion', 'Makanan', 'Minuman'], n_transaksi),
    'Jumlah': np.random.randint(1, 10, n_transaksi),
    'Harga_Satuan': np.random.randint(10000, 1000000, n_transaksi),
    'Kota': np.random.choice(['Jakarta', 'Bandung', 'Surabaya', 'Medan'], n_transaksi),
    'Metode_Pembayaran': np.random.choice(['Cash', 'Credit Card', 'E-Wallet'], n_transaksi)
}

# Membuat DataFrame
df_sales = pd.DataFrame(data_penjualan)

# Menambahkan kolom Total Penjualan
df_sales['Total_Penjualan'] = df_sales['Jumlah'] * df_sales['Harga_Satuan']

print("Data Penjualan:")
print(df_sales.head())
print("\nInformasi Dataset:")
print(df_sales.info())

Data Penjualan:
     Tanggal ID_Produk Kategori  Jumlah  Harga_Satuan      Kota  \
0 2024-01-01      P007  Makanan       8        100272     Medan   
1 2024-01-02      P020  Minuman       9         48467  Surabaya   
2 2024-01-03      P015  Minuman       4        819760   Jakarta   
3 2024-01-04      P011  Fashion       1        887844   Bandung   
4 2024-01-05      P008  Makanan       1        490047  Surabaya   

  Metode_Pembayaran  Total_Penjualan  
0              Cash           802176  
1          E-Wallet           436203  
2       Credit Card          3279040  
3       Credit Card           887844  
4       Credit Card           490047  

Informasi Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Tanggal            100 non-null    datetime64[ns]
 1   ID_Produk          100 non-null    object        
 2  

### Analisis Data Penjualan

Mari kita lakukan beberapa analisis menggunakan berbagai teknik grouping:

In [41]:
# 1. Total penjualan per kategori
print("Total Penjualan per Kategori:")
kategori_sales = df_sales.groupby('Kategori')['Total_Penjualan'].agg(['sum', 'mean', 'count']).round(2)
print(kategori_sales)
print("\n" + "-"*50)

# 2. Analisis per kota dan metode pembayaran
print("\nAnalisis per Kota dan Metode Pembayaran:")
kota_payment = pd.pivot_table(df_sales, 
                            values='Total_Penjualan',
                            index='Kota',
                            columns='Metode_Pembayaran',
                            aggfunc='sum',
                            fill_value=0).round(2)
print(kota_payment)
print("\n" + "-"*50)

# 3. Trend penjualan per bulan
df_sales['Bulan'] = df_sales['Tanggal'].dt.to_period('M')
print("\nTrend Penjualan Bulanan per Kategori:")
monthly_trend = df_sales.groupby(['Bulan', 'Kategori'])['Total_Penjualan'].sum().round(2)
print(monthly_trend)

Total Penjualan per Kategori:
                 sum        mean  count
Kategori                               
Elektronik  68905172  2756206.88     25
Fashion     50896998  2423666.57     21
Makanan     60071857  1877245.53     32
Minuman     45655206  2075236.64     22

--------------------------------------------------

Analisis per Kota dan Metode Pembayaran:
Metode_Pembayaran      Cash  Credit Card  E-Wallet
Kota                                              
Bandung            28256670     27203294  10714755
Jakarta            13579897     12913003  14361884
Medan              25556576     30458651  13386549
Surabaya           19279968     15400047  14417939

--------------------------------------------------

Trend Penjualan Bulanan per Kategori:
Bulan    Kategori  
2024-01  Elektronik    14406549
         Fashion        9305043
         Makanan       19474248
         Minuman       10576263
2024-02  Elektronik    29359135
                         ...   
2024-03  Minuman       1404

## 4. Latihan

Sekarang saatnya untuk berlatih! Gunakan dataset penjualan di atas untuk menyelesaikan soal-soal berikut:

1. Hitung rata-rata penjualan harian untuk setiap kategori produk
2. Temukan produk dengan penjualan tertinggi di setiap kota
3. Analisis distribusi metode pembayaran untuk setiap kategori produk
4. Hitung total penjualan mingguan dan bandingkan antar kategori
5. Identifikasi tren penjualan (naik/turun) untuk setiap kategori

Berikut adalah template kode untuk memulai latihan:

In [42]:
# Template untuk latihan

# Soal 1: Rata-rata penjualan harian per kategori
# Hint: Gunakan groupby dengan Tanggal dan Kategori

# Soal 2: Produk dengan penjualan tertinggi per kota
# Hint: Gunakan groupby dengan Kota dan ID_Produk, kemudian ambil yang tertinggi

# Soal 3: Distribusi metode pembayaran per kategori
# Hint: Gunakan crosstab atau pivot_table

# Soal 4: Total penjualan mingguan per kategori
# Hint: Tambahkan kolom minggu menggunakan dt.isocalendar().week

# Soal 5: Tren penjualan per kategori
# Hint: Hitung perubahan persentase dari periode ke periode

### Solusi Latihan

Berikut adalah solusi untuk latihan di atas. Cobalah untuk mengerjakan sendiri terlebih dahulu sebelum melihat solusi!

In [44]:
# Solusi Soal 1: Rata-rata penjualan harian per kategori
print("Rata-rata Penjualan Harian per Kategori:")
daily_avg = df_sales.groupby(['Tanggal', 'Kategori'])['Total_Penjualan'].mean().round(2)
print(daily_avg.head(10))
print("\n" + "-"*50)

# Solusi Soal 2: Produk dengan penjualan tertinggi per kota
print("\nProduk dengan Penjualan Tertinggi per Kota:")
top_products = df_sales.loc[df_sales.groupby('Kota')['Total_Penjualan'].idxmax()]
print(top_products[['Kota', 'ID_Produk', 'Total_Penjualan']])
print("\n" + "-"*50)

# Solusi Soal 3: Distribusi metode pembayaran per kategori
print("\nDistribusi Metode Pembayaran per Kategori:")
payment_dist = pd.crosstab(df_sales['Kategori'], 
                          df_sales['Metode_Pembayaran'], 
                          values=df_sales['Total_Penjualan'], 
                          aggfunc='sum').round(2)
print(payment_dist)
print("\n" + "-"*50)

# Solusi Soal 4: Total penjualan mingguan per kategori
df_sales['Minggu'] = df_sales['Tanggal'].dt.isocalendar().week
print("\nTotal Penjualan Mingguan per Kategori:")
weekly_sales = df_sales.groupby(['Minggu', 'Kategori'])['Total_Penjualan'].sum().round(2)
print(weekly_sales)
print("\n" + "-"*50)

# Solusi Soal 5: Tren penjualan per kategori
print("\nTren Penjualan per Kategori:")
df_sales['Bulan'] = df_sales['Tanggal'].dt.to_period('M')
monthly_trend = df_sales.groupby(['Bulan', 'Kategori'])['Total_Penjualan'].sum().round(2)
monthly_pct_change = monthly_trend.groupby(level=1).pct_change() * 100
print("Perubahan Persentase Bulanan:")
print(monthly_pct_change.round(2))

Rata-rata Penjualan Harian per Kategori:
Tanggal     Kategori  
2024-01-01  Makanan        802176.0
2024-01-02  Minuman        436203.0
2024-01-03  Minuman       3279040.0
2024-01-04  Fashion        887844.0
2024-01-05  Makanan        490047.0
2024-01-06  Makanan       2677388.0
2024-01-07  Elektronik     482097.0
2024-01-08  Makanan       1899806.0
2024-01-09  Elektronik     415173.0
2024-01-10  Makanan        449430.0
Name: Total_Penjualan, dtype: float64

--------------------------------------------------

Produk dengan Penjualan Tertinggi per Kota:
        Kota ID_Produk  Total_Penjualan
12   Bandung      P002          6723968
60   Jakarta      P004          6252288
57     Medan      P012          7472340
79  Surabaya      P012          7604776

--------------------------------------------------

Distribusi Metode Pembayaran per Kategori:
Metode_Pembayaran      Cash  Credit Card  E-Wallet
Kategori                                          
Elektronik         28073304     27456249  1

## Kesimpulan

Dalam tutorial ini, kita telah mempelajari:

1. Dasar-dasar grouping dengan Pandas
   - Penggunaan `groupby()`
   - Fungsi agregasi
   - Multiple grouping

2. Teknik-teknik agregasi lanjutan
   - Custom aggregation
   - Transform
   - Pivot tables dan crosstab

3. Analisis data real-world
   - Analisis penjualan
   - Tren temporal
   - Distribusi kategorikal

4. Tips dan Best Practices
   - Gunakan fungsi agregasi yang sesuai dengan tipe data
   - Manfaatkan multiple grouping untuk analisis yang lebih mendalam
   - Perhatikan format output dan pembulatan angka

Dengan pemahaman ini, Anda dapat melakukan analisis data yang lebih kompleks dan menghasilkan insights yang bermakna dari dataset Anda.